# [Go to "Styling" in pandas docs](https://pandas.pydata.org/pandas-docs/stable/user_guide/style.html)

In [1]:
import pandas as pd, numpy as np

The `DataFrame.style` property returns a `Styler` object, which has useful methods for formatting and displaying `DataFrames`.

Styling is accomplished using **CSS**: through functions that take scalars, `DataFrames` or `Series`, and return like-indexed `DataFrames` or `Series` with CSS `"attribute: value"` pairs for the values.

>`Styler` has a `_repr_html_` method defined on it so tables are rendered automatically.

In [2]:
df = pd.DataFrame(np.random.standard_normal(20).reshape(5, 4), columns=list('ABCD'))

print(type(df.style))

df.style

<class 'pandas.io.formats.style.Styler'>


,A,B,C,D
0,-0.089363,-1.647474,0.260742,1.051696
1,-0.999118,0.798054,-0.140809,-0.108498
2,-0.349575,0.317873,-0.451232,0.073592
3,0.700035,-0.828489,0.148047,-0.748481
4,-2.209105,1.153856,-2.571906,-1.013893


>If you want the actual HTML back for further processing or for writing to file, call the `.render()` method, which returns a `string`.

In [3]:
df[:2].style.render()

'<style  type="text/css" >\n</style><table id="T_d5acfaaf_b6bd_11ea_a5c1_b068e6d46a5d" ><thead>    <tr>        <th class="blank level0" ></th>        <th class="col_heading level0 col0" >A</th>        <th class="col_heading level0 col1" >B</th>        <th class="col_heading level0 col2" >C</th>        <th class="col_heading level0 col3" >D</th>    </tr></thead><tbody>\n                <tr>\n                        <th id="T_d5acfaaf_b6bd_11ea_a5c1_b068e6d46a5dlevel0_row0" class="row_heading level0 row0" >0</th>\n                        <td id="T_d5acfaaf_b6bd_11ea_a5c1_b068e6d46a5drow0_col0" class="data row0 col0" >-0.089363</td>\n                        <td id="T_d5acfaaf_b6bd_11ea_a5c1_b068e6d46a5drow0_col1" class="data row0 col1" >-1.647474</td>\n                        <td id="T_d5acfaaf_b6bd_11ea_a5c1_b068e6d46a5drow0_col2" class="data row0 col2" >0.260742</td>\n                        <td id="T_d5acfaaf_b6bd_11ea_a5c1_b068e6d46a5drow0_col3" class="data row0 col3" >1.051696</td>\n

# 1. Building Styles

Style functions should return strings with one or more CSS `attribute: value` pairs delimited by semicolons. Use:

- `Styler.applymap(func)` for elementwise styles

- `Styler.apply(func, axis=0)` for columnwise styles

- `Styler.apply(func, axis=1)` for rowwise styles

- `Styler.apply(func, axis=None)` for tablewise styles

### 1.1 `Styler.applymap` - elementwise.

In [4]:
def color_negatives_red(val):
    """
    Converts -ve values' color to red. 
    """
    color = 'red' if val < 0 else 'black'
    return f'color: {color}'


df.style.applymap(color_negatives_red)

,A,B,C,D
0,-0.089363,-1.647474,0.260742,1.051696
1,-0.999118,0.798054,-0.140809,-0.108498
2,-0.349575,0.317873,-0.451232,0.073592
3,0.700035,-0.828489,0.148047,-0.748481
4,-2.209105,1.153856,-2.571906,-1.013893


### 1.2 `Styler.apply` - column-wise(axis=0)

In [5]:
def highlight_max(s):
    '''
    Highlight the maximum in a Series or DataFrame colums/rows yellow
    '''
    is_max = s == s.max()
    return ['background-color: yellow' if v else '' for v in is_max]

df.style.apply(highlight_max)  # column-wise (default: axis=0)

,A,B,C,D
0,-0.089363,-1.647474,0.260742,1.051696
1,-0.999118,0.798054,-0.140809,-0.108498
2,-0.349575,0.317873,-0.451232,0.073592
3,0.700035,-0.828489,0.148047,-0.748481
4,-2.209105,1.153856,-2.571906,-1.013893


### 1.3 `Styler.apply` -  row-wise(axis=1)

In [6]:
df.style.apply(highlight_max, axis=1)  # row-wise

,A,B,C,D
0,-0.089363,-1.647474,0.260742,1.051696
1,-0.999118,0.798054,-0.140809,-0.108498
2,-0.349575,0.317873,-0.451232,0.073592
3,0.700035,-0.828489,0.148047,-0.748481
4,-2.209105,1.153856,-2.571906,-1.013893


### 1.4 `Styler.apply` - table-wise(axis=None)

In [7]:
def enlarge_max_whole_table(s):
    '''
    Highlight the maximum in a Series or DataFrame colums/rows yellow
    '''
    foo = lambda x: ['font-size: 2em' if y == s.max().max() \
                     else '' for y in x]
    return s.apply(foo)

# applying style function to whole dataframe (axis=None)
df.style.apply(enlarge_max_whole_table, axis=None)  

,A,B,C,D
0,-0.089363,-1.647474,0.260742,1.051696
1,-0.999118,0.798054,-0.140809,-0.108498
2,-0.349575,0.317873,-0.451232,0.073592
3,0.700035,-0.828489,0.148047,-0.748481
4,-2.209105,1.153856,-2.571906,-1.013893


# 2. Slicing (Specifying rows/columns)

Both `Styler.apply` and `Styler.applymap` accept a **`subset`** keyword, which allows you to apply styles to specific rows or columns, without having to code that logic into your style function.

In [8]:
df.style.apply(highlight_max, subset=['B', 'D'])\
.applymap(color_negatives_red, subset=['A', 'C'])

,A,B,C,D
0,-0.089363,-1.647474,0.260742,1.051696
1,-0.999118,0.798054,-0.140809,-0.108498
2,-0.349575,0.317873,-0.451232,0.073592
3,0.700035,-0.828489,0.148047,-0.748481
4,-2.209105,1.153856,-2.571906,-1.013893


# 3. Formating Display Values

The display value can be controlled using `Styler.format`.

Cells can be formatted according to a [format spec string][1] or a callable that takes a single value and returns a string.

[1]: https://docs.python.org/3/library/string.html#format-specification-mini-language

In [9]:
df.iloc[:2, 1] = np.nan
df.style.format("{:.2%}")

,A,B,C,D
0,-8.94%,nan%,26.07%,105.17%
1,-99.91%,nan%,-14.08%,-10.85%
2,-34.96%,31.79%,-45.12%,7.36%
3,70.00%,-82.85%,14.80%,-74.85%
4,-220.91%,115.39%,-257.19%,-101.39%


> You can use a `dict` to specify columns

In [10]:
df.style.format({'A': "{: ^.3f}", 'D': '{:>10}'})

,A,B,C,D
0,-0.089,nan,0.260742,1.0516961875537978
1,-0.999,nan,-0.140809,-0.1084976292846756
2,-0.350,0.317873,-0.451232,0.07359206329338162
3,0.700,-0.828489,0.148047,-0.7484808038428142
4,-2.209,1.153856,-2.571906,-1.0138927241246913


> Passing a callable (or dictionary of callables) for more flexible handling

In [11]:
df.style.format({"C": lambda x: f"±{abs(x):.2f}"})

,A,B,C,D
0,-0.089363,nan,±0.26,1.051696
1,-0.999118,nan,±0.14,-0.108498
2,-0.349575,0.317873,±0.45,0.073592
3,0.700035,-0.828489,±0.15,-0.748481
4,-2.209105,1.153856,±2.57,-1.013893


>You can format the text displayed for missing values using `na_rep`

In [12]:
df.style.format("{:.2%}", na_rep="-")

,A,B,C,D
0,-8.94%,-,26.07%,105.17%
1,-99.91%,-,-14.08%,-10.85%
2,-34.96%,31.79%,-45.12%,7.36%
3,70.00%,-82.85%,14.80%,-74.85%
4,-220.91%,115.39%,-257.19%,-101.39%


> Formatting techniques can be used in combination with styling:

In [13]:
df.style.highlight_max().format(None, na_rep="_")

,A,B,C,D
0,-0.089363,_,0.260742,1.051696
1,-0.999118,_,-0.140809,-0.108498
2,-0.349575,0.317873,-0.451232,0.073592
3,0.700035,-0.828489,0.148047,-0.748481
4,-2.209105,1.153856,-2.571906,-1.013893


# 4. Builtin Styles

In [14]:
attributes = []
[None if x.startswith('_') else attributes.append(x) for x in dir(df.style)]
n = len(attributes)
for x in range(0, 3 * (n // 3), 3):
    print(f'{attributes[x]:20} {attributes[x+1]:20} {attributes[x+2]:20}')
    
for x in range(3 * (n // 3), 3 * (n // 3) + divmod(n, 3)[1]):
    print(f'{attributes[x]:20}', sep='')

apply                applymap             background_gradient 
bar                  caption              cell_ids            
clear                columns              ctx                 
data                 env                  export              
format               from_custom_template hidden_columns      
hidden_index         hide_columns         hide_index          
highlight_max        highlight_min        highlight_null      
index                loader               na_rep              
pipe                 precision            render              
set_caption          set_na_rep           set_precision       
set_properties       set_table_attributes set_table_styles    
set_uuid             table_attributes     table_styles        
template             to_excel             use                 
uuid                
where               


In [15]:
df.style.highlight_null(null_color='cyan')

,A,B,C,D
0,-0.089363,nan,0.260742,1.051696
1,-0.999118,nan,-0.140809,-0.108498
2,-0.349575,0.317873,-0.451232,0.073592
3,0.700035,-0.828489,0.148047,-0.748481
4,-2.209105,1.153856,-2.571906,-1.013893


>You can create “heatmaps” with the `background_gradient` method

In [16]:
df.style.background_gradient()

,A,B,C,D
0,-0.089363,nan,0.260742,1.051696
1,-0.999118,nan,-0.140809,-0.108498
2,-0.349575,0.317873,-0.451232,0.073592
3,0.700035,-0.828489,0.148047,-0.748481
4,-2.209105,1.153856,-2.571906,-1.013893


In [17]:
df.style.highlight_max()

,A,B,C,D
0,-0.089363,nan,0.260742,1.051696
1,-0.999118,nan,-0.140809,-0.108498
2,-0.349575,0.317873,-0.451232,0.073592
3,0.700035,-0.828489,0.148047,-0.748481
4,-2.209105,1.153856,-2.571906,-1.013893


In [18]:
df.style.highlight_min()

,A,B,C,D
0,-0.089363,nan,0.260742,1.051696
1,-0.999118,nan,-0.140809,-0.108498
2,-0.349575,0.317873,-0.451232,0.073592
3,0.700035,-0.828489,0.148047,-0.748481
4,-2.209105,1.153856,-2.571906,-1.013893


In [19]:
df.style.set_properties(**{'background-color': 'grey',
                           'color': 'orange'})

,A,B,C,D
0,-0.089363,nan,0.260742,1.051696
1,-0.999118,nan,-0.140809,-0.108498
2,-0.349575,0.317873,-0.451232,0.073592
3,0.700035,-0.828489,0.148047,-0.748481
4,-2.209105,1.153856,-2.571906,-1.013893


In [20]:
df['E']=range(1, 6)
df.style.bar(subset=['A', 'C', 'E'], color='skyblue')

,A,B,C,D,E
0,-0.089363,nan,0.260742,1.051696,1
1,-0.999118,nan,-0.140809,-0.108498,2
2,-0.349575,0.317873,-0.451232,0.073592,3
3,0.700035,-0.828489,0.148047,-0.748481,4
4,-2.209105,1.153856,-2.571906,-1.013893,5


In [21]:
df.style.bar(subset=['A', 'B'], align='mid', color=['crimson', 'skyblue'])

,A,B,C,D,E
0,-0.089363,nan,0.260742,1.051696,1
1,-0.999118,nan,-0.140809,-0.108498,2
2,-0.349575,0.317873,-0.451232,0.073592,3
3,0.700035,-0.828489,0.148047,-0.748481,4
4,-2.209105,1.153856,-2.571906,-1.013893,5


# 5. Sharing Styles

>You can access an existing style using the `export` method, and apply it to a different dataset with the `use` method.

In [22]:
style1 = df.style.apply(enlarge_max_whole_table, axis=None)
style1

,A,B,C,D,E
0,-0.089363,nan,0.260742,1.051696,1
1,-0.999118,nan,-0.140809,-0.108498,2
2,-0.349575,0.317873,-0.451232,0.073592,3
3,0.700035,-0.828489,0.148047,-0.748481,4
4,-2.209105,1.153856,-2.571906,-1.013893,5


In [23]:
df2 = -df
df2.style.use(style1.export())

,A,B,C,D,E
0,0.089363,nan,-0.260742,-1.051696,-1.000000
1,0.999118,nan,0.140809,0.108498,-2.000000
2,0.349575,-0.317873,0.451232,-0.073592,-3.000000
3,-0.700035,0.828489,-0.148047,0.748481,-4.000000
4,2.209105,-1.153856,2.571906,1.013893,-5.000000


# 6. Options independent of the data

### 6.1 Precision

In [24]:
df.style.set_precision(3)

,A,B,C,D,E
0,-0.089,nan,0.261,1.052,1
1,-0.999,nan,-0.141,-0.108,2
2,-0.350,0.318,-0.451,0.074,3
3,0.700,-0.828,0.148,-0.748,4
4,-2.209,1.154,-2.572,-1.014,5


### 6.2 Captions

In [25]:
df.style.set_caption('Some useful caption')

,A,B,C,D,E
0,-0.089363,nan,0.260742,1.051696,1
1,-0.999118,nan,-0.140809,-0.108498,2
2,-0.349575,0.317873,-0.451232,0.073592,3
3,0.700035,-0.828489,0.148047,-0.748481,4
4,-2.209105,1.153856,-2.571906,-1.013893,5


### 6.3 Table Styles

In [26]:
from IPython.display import HTML

def hover(hover_color="yellow"):
    return {'selector': "tr:hover",
            'props': [("background-color", f"{hover_color}")]}

styles = [
    hover(),
    {'selector': "th", 'props': [("font-size", "150%"),
                                 ("text-align", "center"),
                                 ("color", "slategrey")]},
    {'selector': "caption", 'props': [("caption-side", "bottom"),
                                      ("font-style", "italic")]}
]
html = (df.style.set_table_styles(styles)
          .set_caption("Hover to highlight."))
html

,A,B,C,D,E
0,-0.089363,nan,0.260742,1.051696,1
1,-0.999118,nan,-0.140809,-0.108498,2
2,-0.349575,0.317873,-0.451232,0.073592,3
3,0.700035,-0.828489,0.148047,-0.748481,4
4,-2.209105,1.153856,-2.571906,-1.013893,5


### 6.4 Missing Values

In [27]:
df.style.set_na_rep('\ | /').highlight_null("yellow")

,A,B,C,D,E
0,-0.089363,\ | /,0.260742,1.051696,1
1,-0.999118,\ | /,-0.140809,-0.108498,2
2,-0.349575,0.317873,-0.451232,0.073592,3
3,0.700035,-0.828489,0.148047,-0.748481,4
4,-2.209105,1.153856,-2.571906,-1.013893,5


### 6.5 Hiding the Index or Columns

In [28]:
df.style.hide_index()

A,B,C,D,E
-0.089363,nan,0.260742,1.051696,1
-0.999118,nan,-0.140809,-0.108498,2
-0.349575,0.317873,-0.451232,0.073592,3
0.700035,-0.828489,0.148047,-0.748481,4
-2.209105,1.153856,-2.571906,-1.013893,5


In [29]:
df.style.hide_columns(['B','E'])

,A,C,D
0,-0.089363,0.260742,1.051696
1,-0.999118,-0.140809,-0.108498
2,-0.349575,-0.451232,0.073592
3,0.700035,0.148047,-0.748481
4,-2.209105,-2.571906,-1.013893


# 7. Interesting Features ("Fun Stuff")

In [30]:
from IPython.html import widgets
@widgets.interact
def f(h_neg=(0, 359, 1), h_pos=(0, 359), s=(0., 99.9), l=(0., 99.9)):
    return df.style.background_gradient(
        
    )

/home/tim/Projects/learning-pandas/venv/lib/python3.6/site-packages/IPython/html.py:14: ShimWarning: The `IPython.html` package has been deprecated since IPython 4.0. You should import from `notebook` instead. `IPython.html.widgets` has moved to `ipywidgets`.
  "`IPython.html.widgets` has moved to `ipywidgets`.", ShimWarning)


interactive(children=(IntSlider(value=179, description='h_neg', max=359), IntSlider(value=179, description='h_…

In [31]:
def magnify():
    return [dict(selector="th",
                 props=[("font-size", "4pt")]),
            dict(selector="td",
                 props=[('padding', "0em 0em")]),
            dict(selector="th:hover",
                 props=[("font-size", "12pt")]),
            dict(selector="tr:hover td:hover",
                 props=[('max-width', '200px'),
                        ('font-size', '12pt')])
]

df = pd.DataFrame(np.random.rand(25, 25))
df.style.background_gradient( axis=1)\
    .set_properties(**{'max-width': '80px', 'font-size': '1pt'})\
    .set_caption("Hover to magnify")\
    .set_precision(2)\
    .set_table_styles(magnify())

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24
0,0.10,0.78,0.31,0.87,0.01,0.81,0.06,0.24,0.95,0.38,0.60,0.81,0.18,0.40,0.33,0.88,0.79,0.32,0.70,0.75,0.14,0.86,0.54,0.54,0.31
1,0.11,0.99,0.33,0.13,0.98,0.16,0.16,0.20,0.12,0.34,0.18,0.95,0.98,0.11,0.80,0.57,0.56,1.00,0.69,0.68,0.93,0.39,0.91,0.09,0.20
2,0.93,0.14,0.65,0.48,0.39,0.46,0.25,0.22,0.70,0.78,0.34,0.83,0.63,0.72,0.96,0.93,0.19,0.58,0.88,0.07,0.05,0.24,0.13,0.52,0.67
3,0.29,0.77,0.58,0.78,0.38,0.15,0.20,0.66,0.75,0.03,0.41,0.63,0.18,0.74,0.16,0.73,0.62,0.55,0.70,0.56,0.73,0.15,0.29,0.50,0.10
4,0.62,0.44,0.94,0.04,0.47,0.55,0.04,0.15,0.50,0.60,0.98,0.33,0.10,0.02,0.17,0.60,0.60,0.90,0.14,0.55,0.66,0.80,0.44,0.11,0.82
5,0.08,0.90,0.91,0.41,0.40,0.70,0.37,0.66,0.54,1.00,0.18,0.08,0.10,0.69,0.44,0.47,0.11,0.28,0.45,0.46,0.44,0.81,0.67,0.09,0.58
6,0.97,0.27,0.97,0.92,0.75,0.18,0.45,0.93,0.85,0.04,0.78,0.16,0.76,0.10,0.32,0.87,0.85,0.12,0.59,0.39,0.92,0.60,0.68,0.77,0.74
7,0.95,0.87,0.57,0.10,0.53,0.51,0.73,0.96,0.21,0.34,0.74,0.05,0.65,0.05,0.37,0.10,0.39,0.86,0.91,0.26,0.60,0.86,0.97,0.41,0.63
8,0.54,0.92,0.82,0.05,0.20,0.36,0.68,0.95,0.82,0.76,0.46,0.33,0.09,0.15,0.07,0.00,0.28,0.92,0.08,0.99,0.84,0.44,0.25,0.95,0.70
9,0.28,0.50,0.97,0.80,0.78,0.35,0.60,0.58,0.66,0.77,0.00,0.76,0.16,0.12,0.61,0.12,0.21,0.60,0.89,0.36,0.20,0.62,0.39,0.64,0.12


# 8. Exporting to Excel

- This feature is still experimenatal and subject to changes/enhancements.
- Some support is available for exporting styled `DataFrames` to Excel worksheets using the `OpenPyXL` or `XlsxWriter` engines.
- Only CSS2 named colors and hex colors of the form `#rgb` or `#rrggbb` are currently supported. Properties available include:
```
    background-color       white-space: nowrap

    border-style, border-width, border-color and their {top, right, bottom, left variants}

    color      font-family      font-style     font-weight

    text-align       text-decoration       vertical-align
```

In [32]:
df.style.apply(highlight_max)\
    .to_excel('styled.xlsx', engine='openpyxl')